**Demo Setup...run the first and second cell before the session**

In [2]:
$ENV:KUBECONFIG='/Users/aen/.kube/arc-config'
kubectl config get-contexts
kubectl config set-context kubernetes-admin
kubectl get nodes

CURRENT   NAME                          CLUSTER      AUTHINFO           NAMESPACE
          kubernetes-admin                                              
*         kubernetes-admin@kubernetes   kubernetes   kubernetes-admin   


Context "kubernetes-admin" modified.


NAME    STATUS   ROLES    AGE   VERSION
arc01   Ready    master   22d   v1.18.3


In [ ]:
$ENV:AZDATA_USERNAME='arcadmin'
$ENV:AZDATA_PASSWORD='S0methingS@Str0ng!'

azdata login --endpoint https://172.16.94.14:30080 --namespace arc

azdata arc sql mi create -n sqldemo01

Logged in successfully to `https://172.16.94.14:30080` in namespace `arc`. Setting active context to `arc`.


ERROR: Arc SQL managed instance `sqldemo01` already exists in namespace `arc`.


**First up, let's log into our Azure Arc Data Controller using the Azure Data CLI ( `azdata` )**
The Kubernetes cluster we're using is local on my laptop here and we will integrate this with Azure using Indirect Connectivity Mode

In [3]:
$ENV:AZDATA_USERNAME='arcadmin'
$ENV:AZDATA_PASSWORD='S0methingS@Str0ng!'

azdata login --endpoint https://172.16.94.14:30080 --namespace arc

Logged in successfully to `https://172.16.94.14:30080` in namespace `arc`. Setting active context to `arc`.


**As with any CLI tool, your first stop should be the help :) Get familiar with the syntax and options available.**

In [ ]:
azdata arc --help


Group
    azdata arc : Commands for using Azure Arc for Azure data services.

Subgroups:
    dc            : Create, delete, and manage data controllers.
    postgres      : Create, delete, and managed Azure Arc enabled PostgreSQL Hyperscale server
                    groups.
    resource-kind : Resource-kind commands to define and template custom resources on your cluster.
    sql           : Create, delete, and manage SQL resources.



**I've  already deployed one Managed Instance and one PostgreSQL Hyperscale instance into our Arc Data Services on premises environment. To get connection information we can use `azdata`.**

In [ ]:
azdata arc sql --help
azdata arc sql mi list


Group
    azdata arc sql : Create, delete, and manage SQL resources.

Subgroups:
    endpoint : View and manage SQL endpoints.
    mi       : Create, delete, and manage SQL managed instance.



ExternalEndpoint    Name       Replicas    State
------------------  ---------  ----------  -------
172.16.94.14,31501  sqldemo01  1/1         Ready


In [ ]:
kubectl get services --namespace arc

NAME                      TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                                       AGE
controldb-svc             ClusterIP   10.105.201.254   <none>        1433/TCP,8311/TCP,8411/TCP                    3h24m
controller-svc            ClusterIP   10.98.89.150     <none>        443/TCP,8311/TCP,8301/TCP,8411/TCP,8401/TCP   3h24m
controller-svc-external   NodePort    10.101.71.249    <none>        8443:30080/TCP                                3h24m
logsdb-svc                ClusterIP   10.109.71.166    <none>        9200/TCP,8300/TCP,8400/TCP                    3h23m
logsui-svc                ClusterIP   10.103.245.221   <none>        5601/TCP,8300/TCP,8400/TCP                    3h23m
metricsdb-svc             ClusterIP   10.99.122.152    <none>        8086/TCP,8300/TCP,8400/TCP                    3h23m
metricsdc-svc             ClusterIP   10.104.176.66    <none>        8300/TCP,8400/TCP                             3h23m
metricsui-svc             ClusterI

**Let's get the Port (NodePort) for our deployed Managed Instance which is backed by the Service `sqldemo01-external-svc`**

**Here is an example of being able to use kubernetes native tooling**

In [ ]:
$NodePort=kubectl get services --namespace arc sqldemo01-external-svc -o jsonpath='{ .spec.ports[0].nodePort }'
sqlcmd -S 172.16.94.14,$NodePort -U arcadmin -P 'S0methingS@Str0ng!' -Q 'SELECT @@VERSION'

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft Azure SQL Database Managed Instance - Azure Arc - 15.0.2000.532 (X64) 
	Sep 10 2020 03:08:27 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.7 LTS) <X64>                                                                                           

(1 rows affected)


**Let's check out the help command for creating a Managed Instance**

In [ ]:
azdata arc sql mi create --help


Command
    azdata arc sql mi create : Create a SQL managed instance.
        To set the password of the SQL managed instance, please set the environment variable
        AZDATA_PASSWORD.

Arguments
    --name -n            [Required] : The name of the SQL managed instance.
    --cores-limit -cl               : The cores limit of the managed instance as an integer.
    --cores-request -cr             : The request for cores of the managed instance as an integer.
    --memory-limit -ml              : The limit of the capacity of the managed instance as an
                                      integer.
    --memory-request -mr            : The request for the capcity of the managed instance as an
                                      integer amount of memory in GBs.
    --no-external-endpoint          : If specified, no external service will be created. Otherwise,
                                      an external service will be created using the same service
                           

**We can create one with minimal options which will get you a Managed Instance with no CPU or Memory limits and Requests of 100m and 100Mi and a single disk from your default StorageClass. Takes about 1 minute with containers pre-pulled.**

In [ ]:
azdata arc sql mi create --name sqldemo02 

Using AZDATA_USERNAME environment variable for `sqldemo02` username.
Using AZDATA_PASSWORD environment variable for `sqldemo02` password.


**Let's check out our newly deployed Managed Instance. Now we have two unique Service Endpoints.**

In [ ]:
azdata arc sql mi list 

ExternalEndpoint    Name       Replicas    State
------------------  ---------  ----------  --------
172.16.94.14,31501  sqldemo01  1/1         Ready
172.16.94.14,32252  sqldemo02  0/1         Creating


**Now with all of that let's check out some of the management tooling and to do that...let's upload some data into Azure.** 

**You can use environment variables to help build automation around these tools.** 

**Otherwise the prompts are going to be interactive...asking for things like username, namespace, workspace\_id and so on.**

In [5]:
$env:AZDATA_USERNAME="arcadmin"
$env:AZDATA_PASSWORD="S0methingS@Str0ng!"
$env:SUB_ID='fd0c5e48-eea6-4b37-a076-0e23e0df74cb'
$env:SPN_CLIENT_ID='a7b906f4-297f-4aaa-821d-a410e398973e'
$env:SPN_CLIENT_SECRET='9gp9DlIG-vhEwPdggrGMMbmMUnnCplEyRU'
$env:SPN_TENANT_ID='f17cbd44-7697-453e-941c-efe0a4c2d55a'
$env:SPN_AUTHORITY='https://login.microsoftonline.com'
$env:WORKSPACE_ID="31f660e3-cb32-4f1c-97f1-e69896eca90f"
$env:WORKSPACE_SHARED_KEY="TNeKgNLo67T9+8dAN1oDdBzrdwiIdJJ/HD1dwbWRCuWIbyuET/SR/ZpUtkRQvkZxYxrYgB+9niNzXQxAZMQdkg=="

azdata login --endpoint https://172.16.94.14:30080 --namespace arc

azdata arc dc export --type metrics --path metrics.json --force && azdata arc dc upload --path metrics.json
azdata arc dc export --type logs --path logs.json --force && azdata arc dc upload --path logs.json

. {


>> $env:AZDATA_USERNAME="arcadmin"
>> $env:AZDATA_PASSWORD="S0methingS@Str0ng!"
>> $env:SUB_ID='fd0c5e48-eea6-4b37-a076-0e23e0df74cb'
>> $env:SPN_CLIENT_ID='a7b906f4-297f-4aaa-821d-a410e398973e'
>> $env:SPN_CLIENT_SECRET='9gp9DlIG-vhEwPdggrGMMbmMUnnCplEyRU'
>> $env:SPN_TENANT_ID='f17cbd44-7697-453e-941c-efe0a4c2d55a'
>> $env:SPN_AUTHORITY='https://login.microsoftonline.com'
>> $env:WORKSPACE_ID="31f660e3-cb32-4f1c-97f1-e69896eca90f"
>> $env:WORKSPACE_SHARED_KEY="TNeKgNLo67T9+8dAN1oDdBzrdwiIdJJ/HD1dwbWRCuWIbyuET/SR/ZpUtkRQvkZxYxrYgB+9niNzXQxAZMQdkg=="
>> 
>> azdata login --endpoint https://172.16.94.14:30080 --namespace arc
>> 
>> azdata arc dc export --type metrics --path metrics.json --force && azdata arc dc upload --path metrics.json
>> azdata arc dc export --type logs --path logs.json --force && azdata arc dc upload --path logs.json
>> }
>> 


Logged in successfully to `https://172.16.94.14:30080` in namespace `arc`. Setting active context to `arc`.


This option exports metrics of all instances in "arc" to the file: "metrics.json".


	 Successfully got metric: 'CPU Usage'


...

	"arc" is uploaded to Azure "/subscriptions/fd0c5e48-eea6-4b37-a076-0e23e0df74cb/resourcegroups/arc-onprem/providers/Microsoft.AzureArcData/dataControllers/arc"
	"sqldemo01" has been uploaded to Azure "/subscriptions/fd0c5e48-eea6-4b37-a076-0e23e0df74cb/resourcegroups/arc-onprem/providers/Microsoft.AzureArcData/sqlManagedInstances/sqldemo01".


Azure resource_id: /subscriptions/fd0c5e48-eea6-4b37-a076-0e23e0df74cb/resourcegroups/arc-onprem/providers/Microsoft.AzureArcData/sqlManagedInstances/sqldemo01


Metrics upload pushed 85 data points successfully.



This option exports logs of all instances in "arc" to the file: "logs.json".
Start retrieving last 1 days of logs for sqlManagedInstances instance "sqldemo01", this may take some time...


	Collected 0 logs from UTC '2021-03-08 23:40Z' to '2021-03-09 00:52Z'
No log is exported.


ERROR: Cannot find file: "logs.json". Please provide the correct file name and try again


**Management, Monitoring and Logging** - our implementation of Azure Arc Enabled Data Services is Indirectly Connected. The local tools hold the metrics, logging and usage data. One uploaded, you can use tools in Azure to analyze that data. Let's look at the local tools then hop into Azure to look at the tools available there. Install the Azure Arc ADS Extension.
1. Right click Manage to Launch the **Azure Arc Data Controller Dashboard**
    
    - Click on the Managed Instance **sqldemo01** to launch the management dashboard
    - To view logs on-prem click **Kibana Dashboard**
        - Set the date range to 15 days ago and search for **master** and **tempdb**
    - To view Metrics on-prem click **Grafana Dashboard**
        - Review the core SQL Server metrics
2. Back on the main **Azure Arc Data Controller Dashboard**, notice the Connection mode of the Data Controller: **Indirect**. Click on **Open in Azure Portal**.
    
3. Click on the Resource Group name in our case it's **arc-onprem**
    
4. Check out the resources in the Portal
    
    - Click on Metrics, change the **Metric Namespace** to `sql server` **change the Metric** to `Memory Usage`. change the date range to 24 hours.
5. In the Log Analytics workspace and run the following query
    
    - ```
           sqlManagedInstances_logs_CL |  where TimeGenerated > ago(7d)  |  order by TimeGenerated desc 
        ```

In [ ]:
azdata arc sql mi delete --name sqldemo02
azdata arc sql mi delete --name sqldemo01
azdata arc postgres server delete --name postgres01
azdata logout

**Everything deployed inside of Azure Arc Data Services is implemented as a Kubernetes Object...let's check out a few.** 

-   **Pods** - running container based applications inside the Kubernetes cluster
-   **Services** - access endpoints to reach the applications running inside our cluster. There can be cluster internal (ClusterIP) and external Service endpoints (NodePort). Can be integrated with external LoadBalancers if needed.
-   **DaemonSets** - ensure a Pod is running on each node in the cluster. Used by the Metrics data collector in Arc, in our case [Telegraf](https://github.com/influxdata/telegraf "https://github.com/influxdata/telegraf").
-   **ReplicaSets** -  ensure a desired number of Pods are running in the cluster.  Primarily used by management software and web ports for logs and metrics (bootstrapper, control, controlwd, logsui, metricsui and mgmtproxy)
-   **StatefulSets** - ensure a desired number of Pods are running in the cluster. But also provide stable naming on the Pods and cluster service discovery via Cluster DNS. Used by controldb, logsdb, metricsdb and also provisioned Managed Instances and Postgres instances. Enables scaling.

In [ ]:
kubectl get all --namespace arc